<a href="https://colab.research.google.com/github/neuhausmatheus/vicuna/blob/main/Vicuna_13b_gpu_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, let's install the FastChat (Vicuna) repository.

In [ ]:
!git clone https://github.com/thisserand/FastChat.git
%cd FastChat
!pip install -e .

Cloning into 'FastChat'...
remote: Enumerating objects: 816, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 816 (delta 0), reused 2 (delta 0), pack-reused 813
Receiving objects: 100% (816/816), 27.89 MiB | 16.84 MiB/s, done.
Resolving deltas: 100% (492/492), done.
/content/FastChat
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/FastChat
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git (to revision c612628045822f909020f7eb6784c79700813eda) to /tmp/pip-install-f785hyjg/transformers_5e9a26b985944bb087c75a0d38cd6885
  Running command git clone --filter=blob:none --quiet https://github.com/huggin

Second, we will install the GPTQ-for-LLaMA repository which allows us to use GPTQ quantised models.

In [ ]:
!mkdir repositories
%cd repositories
!git clone https://github.com/oobabooga/GPTQ-for-LLaMa.git -b cuda
%cd GPTQ-for-LLaMa
!python setup_cuda.py install
%cd ../..

/content/FastChat/repositories
Cloning into 'GPTQ-for-LLaMa'...
remote: Enumerating objects: 811, done.
remote: Counting objects: 100% (305/305), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 811 (delta 291), reused 284 (delta 284), pack-reused 506
Receiving objects: 100% (811/811), 466.69 KiB | 18.67 MiB/s, done.
Resolving deltas: 100% (493/493), done.
/content/FastChat/repositories/GPTQ-for-LLaMa
running install
/usr/local/lib/python3.9/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
creating quant_cuda.egg-info
writing quant_cuda.egg-info/PKG-INFO
writing dependency_links to quant_cuda.eg

Okay, now we will download the quantised model weights.

In [ ]:
!python download-model.py anon8231489123/vicuna-13b-GPTQ-4bit-128g

100% 606/606 [00:00<00:00, 737kiB/s]
100% 596/596 [00:00<00:00, 616kiB/s]
100% 137/137 [00:00<00:00, 150kiB/s]
100% 33.4k/33.4k [00:00<00:00, 26.2MiB/s]
100% 96.0/96.0 [00:00<00:00, 102kiB/s]
100% 500k/500k [00:00<00:00, 62.5MiB/s]
100% 244/244 [00:00<00:00, 242kiB/s]
100% 7.45G/7.45G [01:40<00:00, 74.0MiB/s]


To be able to run the web UI inside Colab we need virtual terminals. For this, we will use the screen library.

In [ ]:
!sudo apt-get install screen

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  byobu | screenie | iselect ncurses-term
The following NEW packages will be installed:
  screen
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 577 kB of archives.
After this operation, 1,022 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 screen amd64 4.8.0-1ubuntu0.1 [577 kB]
Fetched 577 kB in 2s (343 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected packa

Launch the controler service in the background.

In [ ]:
!screen -S controller -dm python -m fastchat.serve.controller --host "127.0.0.1"

Launch the model worker in the background.

In [ ]:
!screen -S model_worker -dm python -m fastchat.serve.model_worker --model-path anon8231489123/vicuna-13b-GPTQ-4bit-128g --model-name vicuna-gptq --wbits 4 --groupsize 128 --host "127.0.0.1" --worker-address "http://127.0.0.1:21002" --controller-address "http://127.0.0.1:21001"

Wait a little bit (~30 seconds) before you run this cell to make sure that the model worker is up and running. Using the public URL you can interact with the Vicuna model.

In [ ]:
!python -m fastchat.serve.gradio_web_server --controller-url "http://127.0.0.1:21001" --share

2023-04-07 03:43:05 | INFO | gradio_web_server | args: Namespace(host='0.0.0.0', port=None, controller_url='http://127.0.0.1:21001', concurrency_count=4, model_list_mode='once', share=True, moderate=False)
2023-04-07 03:43:05 | INFO | gradio_web_server | Models: ['vicuna-gptq']
2023-04-07 03:43:05 | INFO | gradio_web_server | Namespace(host='0.0.0.0', port=None, controller_url='http://127.0.0.1:21001', concurrency_count=4, model_list_mode='once', share=True, moderate=False)
2023-04-07 03:43:07 | INFO | stdout | Running on local URL:  http://0.0.0.0:7860
2023-04-07 03:43:12 | INFO | stdout | Running on public URL: https://3013e7405e8fc462dc.gradio.live
2023-04-07 03:43:12 | INFO | stdout | 
2023-04-07 03:43:12 | INFO | stdout | This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
2023-04-07 03:46:23 | INFO | gradio_web_server | load_demo. ip: 172.31.34.101. params: {}
2023-04-07 03:46:31 | INFO | gradio_